# Библиотеки

In [6]:
import pandas as pd
import psycopg2

# Актуализация БД

## Подключение к БД

In [7]:
# Функция подключения к базе данных
def connect_to_db():
    try:
        conn = psycopg2.connect(
            dbname="postgres",
            user="postgres",
            password="1234",
            host="localhost",
            port="5432"
        )
        return conn
    except Exception as e:
        print(f"Ошибка подключения к базе данных: {e}")
        return None

## Функция чтения CSV

In [8]:
def read_csv(file_path):
    return pd.read_csv(file_path)

## Функция добавления/обновления статьи

In [9]:
def upsert_article(conn, article_id, title, in_rinc):
    cursor = conn.cursor()
    try:
        cursor.execute("SELECT id FROM elibrary.articles WHERE id = %s", (article_id,))
        existing_article = cursor.fetchone()
        if existing_article:
            cursor.execute("""
                UPDATE elibrary.articles
                SET in_rinc = %s
                WHERE id = %s
            """, (in_rinc, article_id))
        else:
            cursor.execute("""
                INSERT INTO elibrary.articles (id, title, in_rinc)
                VALUES (%s, %s, %s)
            """, (article_id, title, in_rinc))
        conn.commit()
    except Exception as e:
        print(f"Ошибка при обновлении/добавлении статьи: {e}")
        conn.rollback()
    finally:
        cursor.close()

## Функция добавления авторов

In [10]:
def add_authors(conn, article_id, authors_str):
    cursor = conn.cursor()
    try:
        if authors_str.strip().lower() == "нет авторов":
            return
        authors = [author.strip() for author in authors_str.split(",")]
        for author in authors:
            cursor.execute("""
                INSERT INTO elibrary.authors (article_id, author_name, contribution, applied_for_award)
                VALUES (%s, %s, %s, %s)
                ON CONFLICT DO NOTHING
            """, (article_id, author, 0, False))
        conn.commit()
    except Exception as e:
        print(f"Ошибка при добавлении авторов: {e}")
        conn.rollback()
    finally:
        cursor.close()

## Main

In [11]:
if __name__ == "__main__":
    csv_file_path = "final_result.csv"
    main(csv_file_path)

Данные успешно добавлены в базу данных!
